Import Required Dependencies

In [16]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

Let us standardized using headers to make it appear that our request to the server is coming from a legitimate web browser

In [17]:
headers = {
    'Access-Control-Allow-Origin': '*',
    'Access-Control-Allow-Methods': 'GET',
    'Access-Control-Allow-Headers': 'Content-Type',
    'Access-Control-Max-Age': '3600',
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0'
    }

Retrieve wikipage containing our desired data and save store it in a soup object

In [18]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
req = requests.get(url, headers)

soup = BeautifulSoup(req.content, 'html.parser')

Extract the first table (it contains the list of postal code)

In [19]:
table = soup.find('table')

Iterate through all the cell, extract the postal code, borough, neighborhood and save it in list

In [20]:
table_contents=[]
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

Convert the list to dataframe

In [21]:
df=pd.DataFrame(table_contents)

Display the first 5 five rows

In [22]:
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


Print the shape of the dataframe

In [24]:
df.shape

(103, 3)

Get the list of Postal Code

In [34]:
postal_codes = df['PostalCode'].values

Get the Geospacial dataset

In [56]:
df_long_lat = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv')

latitudes = [] 
longitudes = [] 

for postal_code in postal_codes:
    result = df_long_lat[df_long_lat['Postal Code']==postal_code]
    latitude = result['Latitude'].to_string(index=False)
    longitude = result['Longitude'].to_string(index=False)
    latitudes.append(latitude)
    longitudes.append(longitude)
       
df['Latitude'] = latitudes
df['Longitude'] = longitudes


In [57]:
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65426,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
